In [ ]:
"""import random
import tkinter as tk

def affichage():
    """ """"Affiche le nom du joueur qui doit jouer à son tour.""" """
    global tour_joueur
    if tour_joueur == 0:
        tour_joueur += 1 # Création de l'enchainement entre joueur 1 et joueur 2
    return label3.config(text="C'est au tour de " + liste_joueurs[tour_joueur]+ " de jouer")
    tour_joueur -= 1
    return label.config(text="C'est au tour de " + liste_joueurs[tour_joueur]+ " de jouer")
liste_joueurs = (nom_joueur1, nom_joueur2)

tour_joueur = random.randint(0,1)

racine = tk.Tk() # Création de la fenêtre racine
racine.title("JEU PUISSANCE 4") # ajoute un titre
label = tk.Label(racine, text="Prêt à vous affronter au Puissance 4 ?", font=("helvetica", "20")) # création du widget
label.grid(column=3, row=0) # positionnement du widget

nom_joueur1 = input("Quel est ton nom joueur 1 ?") # identification du joueur 1
nom_joueur2 = input("Quel est ton nom joueur 2 ?") # identification du joueur 2


label1 = tk.Label(racine, text="Joueur 2 = " + nom_joueur1, font=("helvetica", "10")) # création du joeur 1
label2 = tk.Label(racine, text="Joueur 2 = " + nom_joueur2, font=("helvetica", "10")) # création du joeur 2
label3 = tk.Label(racine, text=affichage(), font=("helvetica", "10"))
label1.grid(column=0, row=1) # positionnement du nom du joueur 1
label2.grid(column=0, row=2) # positionnement du nom du joeur 2



    









racine = tk.Tk() # Création de la fenêtre racine
racine.mainloop() # Lancement de la boucle principale"""

import random
import tkinter as tk

racine = tk.Tk()
HEIGHT = 500
WIDTH = 500
color = ["yellow","red"]


nom_joueur1 = input("Quel est ton nom joueur 1 ?") # identification du joueur 1
nom_joueur2 = input("Quel est ton nom joueur 2 ?") # identification du joueur 2

def affichage():
    """Affiche le nom du joueur qui doit jouer à son tour."""
    global tour_joueur
    if tour_joueur == 0:
        tour_joueur += 1 # Création de l'enchainement entre joueur 1 et joueur 2
    return label3.config(text="C'est au tour de " + liste_joueurs[tour_joueur]+ " de jouer")
    tour_joueur -= 1
    return label.config(text="C'est au tour de " + liste_joueurs[tour_joueur]+ " de jouer")
liste_joueurs = (nom_joueur1, nom_joueur2)

tour_joueur = random.randint(0,1)

racine = tk.Tk() # Création de la fenêtre racine
racine.title("JEU PUISSANCE 4") # ajoute un titre
label = tk.Label(racine, text="Prêt à vous affronter au Puissance 4 ?", font=("helvetica", "20")) # création du widget
label.grid(column=3, row=0) # positionnement du widget

label1 = tk.Label(racine, text="Joueur 1 = " + nom_joueur1, font=("helvetica", "30")) # création du joeur 1
label2 = tk.Label(racine, text="Joueur 2 = " + nom_joueur2, font=("helvetica", "30")) # création du joeur 2
"""label3 = tk.Label(racine, text=affichage(), font=("helvetica", "10"))"""
label1.grid(column=0, row=1) # positionnement du nom du joueur 1
label2.grid(column=0, row=2) # positionnement du nom du joeur 2
canvas = tk.Canvas(racine, bg="black", height = HEIGHT, width = WIDTH)
canvas.grid()

#  Création des lignes

for i in range(5):
    for j in range(5):
        canvas.create_rectangle((i * 100,j * 100),((i+1) *  100,(j+1) * 100),fill = "red")


cpt = 0
color = ["yellow","blue"]

#  Création des cercles
def jeton(event):
    global cpt
    global liste_coord
    cpt = 1 - cpt
    zone = canvas.find_closest(event.x,event.y)
    a = zone[0] // 5
    cercle = canvas.create_oval(liste_coord[a][0], fill = color[cpt])
    del liste_coord[a][0]


racine.bind("<Button-1>",jeton)


liste_coord = []
for i in range(5):
    listej = [[(i * 100,(4-j) * 100),((i+1) * 100,((4-j)+1) * 100)] for j in range(5)]
    liste_coord.append(listej)



racine.mainloop()
print(liste_coord)